## This notebook generates 2 html-files, that diplay interactive maps for german regional data of:
- annual mean temperature in Celsius
- annual precipitation in mm

(data downloaded from: https://www.dwd.de/DE/klimaumwelt/cdc/cdc_node.html)

In [1]:
import pandas as pd
import plotly.express as px
import json

In [2]:
DATA1 = './data/regional_averages_tm_year.csv'
DATA2 = './data/regional_averages_rr_year.csv'

In [3]:
temp = pd.read_csv(DATA1, sep=';')
prec = pd.read_csv(DATA2, sep=';')

In [4]:
temp.head()

,Jahr,Jahr.1,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland,Unnamed: 19
0,1881,year,7.55,7.54,7.66,6.61,7.49,6.96,7.54,7.54,8.14,7.97,7.12,8.28,6.71,7.46,7.11,6.66,7.31,NaN
1,1882,year,8.99,8.97,8.08,7.33,8.25,8.54,8.88,8.88,9.03,8.55,8.78,8.79,8.12,8.81,8.35,7.77,8.34,NaN
2,1883,year,8.42,8.41,7.77,6.85,7.96,7.95,8.39,8.39,8.71,8.26,8.18,8.51,7.46,8.32,7.87,7.31,7.88,NaN
3,1884,year,9.11,9.10,8.44,7.52,8.58,8.73,9.09,9.10,9.39,8.94,8.86,9.18,8.21,8.94,8.47,7.89,8.57,NaN
4,1885,year,8.40,8.39,7.82,7.04,7.66,7.68,7.94,7.94,8.31,8.01,7.62,8.30,7.73,8.07,7.67,7.16,7.74,NaN


In [5]:
prec.tail()

,Jahr,Jahr.1,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland,Unnamed: 19
136,2017,year,721.4,719.1,957.1,968.1,836.0,801.5,890.0,890.8,875.3,768.0,998.6,910.5,747.3,635.2,699.3,783.0,858.7,NaN
137,2018,year,390.4,390.3,765.0,757.3,576.4,430.1,512.2,512.3,617.7,657.3,577.9,872.7,468.5,352.5,420.6,509.5,586.3,NaN
138,2019,year,505.4,504.6,932.6,860.5,728.8,580.8,718.2,718.5,815.0,756.3,815.3,914.2,602.3,485.5,547.3,627.9,735.0,NaN
139,2020,year,511.1,511.4,816.0,861.0,656.5,564.7,688.8,688.4,740.7,684.1,771.0,864.2,633.4,499.9,568.6,658.4,704.9,NaN
140,2021,year,600.1,600.6,980.9,962.8,743.1,652.4,748.3,748.5,840.7,776.6,802.2,893.8,778.1,582.4,657.3,755.0,801.1,NaN


Colums are beeing renamed to match the names in the geojson-file (ger_4_niedrig.geo.json).
Hamburg - located between Niedersachsen and Schleswig-Holstein - gets their mean-values.

In [6]:
temp = temp.rename(
    columns={'Niedersachsen/Hamburg/Bremen': 'Bremen', 'Brandenburg/Berlin': 'Berlin'})
temp['Hamburg'] = temp[['Niedersachsen', 'Schleswig-Holstein']].mean(axis=1)
temp = temp.drop(columns=['Jahr.1','Thueringen/Sachsen-Anhalt', 'Deutschland', 'Unnamed: 19'])

prec = prec.rename(
    columns={'Niedersachsen/Hamburg/Bremen':'Bremen', 'Brandenburg/Berlin':'Berlin'})
prec['Hamburg'] = prec[['Niedersachsen', 'Schleswig-Holstein']].mean(axis=1)
prec = prec.drop(columns=['Jahr.1','Thueringen/Sachsen-Anhalt', 'Deutschland', 'Unnamed: 19'])
prec

,Jahr,Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen,Hamburg
0,1881,521.3,521.0,828.6,804.9,649.5,494.5,643.5,643.4,792.1,711.2,675.7,776.2,731.0,536.2,655.0,659.60
1,1882,673.6,673.0,1255.2,1085.3,1005.1,568.5,758.8,757.3,1032.1,1078.9,785.4,1210.3,952.3,666.8,882.4,772.10
2,1883,500.6,500.3,870.5,832.4,656.8,518.9,620.4,619.9,754.6,713.6,684.6,819.5,655.9,474.9,593.0,652.50
3,1884,635.4,634.3,776.4,797.2,699.4,600.1,755.2,754.6,808.3,700.3,767.4,769.0,723.8,635.9,672.2,761.30
4,1885,550.2,549.7,990.9,830.2,691.8,537.9,646.7,646.1,734.6,793.3,684.6,925.1,665.0,536.3,615.7,665.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,2017,721.4,719.1,957.1,968.1,836.0,801.5,890.0,890.8,875.3,768.0,998.6,910.5,747.3,635.2,783.0,944.30
137,2018,390.4,390.3,765.0,757.3,576.4,430.1,512.2,512.3,617.7,657.3,577.9,872.7,468.5,352.5,509.5,545.05
138,2019,505.4,504.6,932.6,860.5,728.8,580.8,718.2,718.5,815.0,756.3,815.3,914.2,602.3,485.5,627.9,766.75
139,2020,511.1,511.4,816.0,861.0,656.5,564.7,688.8,688.4,740.7,684.1,771.0,864.2,633.4,499.9,658.4,729.90


In [7]:
temp = temp.melt(id_vars=['Jahr'], var_name='Bundesland', value_name='Mean Temperature in C')
prec = prec.melt(id_vars=['Jahr'], var_name='Bundesland', value_name='Precipitation in mm')
prec

,Jahr,Bundesland,Precipitation in mm
0,1881,Berlin,521.30
1,1882,Berlin,673.60
2,1883,Berlin,500.60
3,1884,Berlin,635.40
4,1885,Berlin,550.20
...,...,...,...
2251,2017,Hamburg,944.30
2252,2018,Hamburg,545.05
2253,2019,Hamburg,766.75
2254,2020,Hamburg,729.90


Load geo-json-file (source: https://github.com/isellsoap/deutschlandGeoJSON)


In [8]:
file = open('./data/ger_4_niedrig.geo.json','r')
ger_json = file.read()
file.close()
ger_json = json.loads(ger_json)
type(ger_json)

dict

Plotly express choropleth-map for temperature data:

In [9]:
fig1 = px.choropleth_mapbox(
    # other styles: https://plotly.com/python/mapbox-layers/
    title='Annual mean temperature in German states',
    mapbox_style='open-street-map',
    data_frame = temp,
    geojson=ger_json,  
    featureidkey='properties.name',     # name of JSON key within the "properties" value that contains country names
    locations='Bundesland', # name of the dataframe column that contains location-names
    color='Mean Temperature in C',                 # name of the dataframe column that contains numerical data we want to display  
    center={"lat": 51.1657, "lon": 10.4515},
    zoom=4.7,
    animation_frame='Jahr',     # name of dataframe column that we want to make frames of
    animation_group='Bundesland',
    color_continuous_scale="plasma",    # other options: https://plotly.com/python/builtin-colorscales/
    range_color=(6, 12),    #color_continuous_midpoint=6,
    opacity=0.8)

In [10]:
fig1.write_html("./output/germany_reg_avg_annual_airtemp_mean.html", include_plotlyjs='cdn')

In [11]:
prec['Precipitation in mm'].min(), prec['Precipitation in mm'].max()

(352.5, 1312.2)

Plotly express choropleth-map for precipitation data:

In [12]:
fig2 = px.choropleth_mapbox(
    title='Annual average precipitation in German states',
    mapbox_style='open-street-map',      # other styles: https://plotly.com/python/mapbox-layers/
    data_frame = prec,                  
    geojson=ger_json,                   
    featureidkey='properties.name',      # name of JSON key within the "properties" value that contains country names
    locations='Bundesland',                    # name of the dataframe column that contains location-names
    color='Precipitation in mm',                       # name of the dataframe column that contains numerical data we want to display
    center={"lat": 51.1657, "lon": 10.4515},
    zoom=4.7,
    animation_frame='Jahr',             # name of dataframe column that we want to make frames of
    animation_group='Bundesland',   
    color_continuous_scale="Blues",    # other options: https://plotly.com/python/builtin-colorscales/
    range_color=(350, 1320), #color_continuous_midpoint=6,
    opacity=0.8)

In [13]:
fig2.write_html(
    "./output/germany_reg_avg_annual_precipitation.html", include_plotlyjs='cdn')


Open the files with the standard browser:

In [18]:
!"./output/germany_reg_avg_annual_airtemp_mean.html"
!"./output/germany_reg_avg_annual_precipitation.html"
